In [1]:
%load_ext autoreload
%autoreload 1

%aimport sheet
%aimport results_comparer
%aimport plot
%aimport settings_handler

JSONDecodeError: Expecting property name enclosed in double quotes: line 8 column 9 (char 172)

In [ ]:
plot.display_image_settings()
#plot.show_final(3)

In [ ]:
results_comparer.display_controls()

In [ ]:
sheet.initial_display()

In [ ]:
settings_handler.display_settings()